In [1]:
# !pip install pandas
# !pip install openpyxl

In [2]:
import pandas as pd
import glob

## 최신 법정동 코드 DF

- 법정동명이 '서울특별시 OO구'인 것만 남기고, 'OO구'로 변환
- 딕셔너리로 생성 : {'법정동명':법정동코드}

In [3]:
new_code_df = pd.read_excel('../dedev_2nd_project/법정동코드 조회자료.xlsx')
new_code_df.head()

,법정동코드,법정동명
0,1100000000,서울특별시
1,1111000000,서울특별시 종로구
2,1111010100,서울특별시 종로구 청운동
3,1111010200,서울특별시 종로구 신교동
4,1111010300,서울특별시 종로구 궁정동


In [4]:
filtered_gucode_df = new_code_df[new_code_df['법정동명'].str.endswith('구')]
new_code_dic = {row['법정동명'].split()[-1]: row['법정동코드'] for _, row in filtered_gucode_df.iterrows()}
print(new_code_dic)
print('서울특별시의 자치구 수 :', len(new_code_dic))

{'종로구': 1111000000, '중구': 1114000000, '용산구': 1117000000, '성동구': 1120000000, '광진구': 1121500000, '동대문구': 1123000000, '중랑구': 1126000000, '성북구': 1129000000, '강북구': 1130500000, '도봉구': 1132000000, '노원구': 1135000000, '은평구': 1138000000, '서대문구': 1141000000, '마포구': 1144000000, '양천구': 1147000000, '강서구': 1150000000, '구로구': 1153000000, '금천구': 1154500000, '영등포구': 1156000000, '동작구': 1159000000, '관악구': 1162000000, '서초구': 1165000000, '강남구': 1168000000, '송파구': 1171000000, '강동구': 1174000000}
서울특별시의 자치구 수 : 25


## 행정동단위 10개 관심집단수 전처리

### 데이터 읽어오기

In [6]:
file_list = glob.glob('../dedev_2nd_project/23_interest_lifedata/*.xlsx')
all_interest_life_df = []

for m, file_path in enumerate(file_list):
    df = pd.read_excel(file_path)
    if m + 1 >= 10:
        df['month'] = str(m+1)
    else:
        df['month'] = '0' + str(m+1)
    all_interest_life_df.append(df)

interest_life_df = pd.concat(all_interest_life_df, ignore_index=True)

### 1. 법정동코드 최신 코드로 변경
    - 코드값 변경
    - 자치구, 행정동명 컬럼 삭제

In [7]:
for key, val in new_code_dic.items():
    interest_life_df.loc[interest_life_df['자치구'] == key, '행정동코드'] = val

In [8]:
del_interest_life_df = interest_life_df.drop(labels=['자치구', '행정동명'], axis=1)
del_interest_life_df.head()

,행정동코드,성별,연령대,총인구,1인가구수,커뮤니케이션이 적은 집단,평일 외출이 적은 집단,휴일 외출이 적은 집단,출근소요시간 및 근무시간이 많은 집단,외출이 매우 적은 집단(전체),외출이 매우 많은 집단,동영상서비스 이용이 많은 집단,생활서비스 이용이 많은 집단,재정상태에 대한 관심집단,외출-커뮤니케이션이 모두 적은 집단(전체),month
0,1111000000,1,20,221.44,53.98,7.97,2.22,5.90,3.48,0.00,4.88,7.72,2.32,1.80,0.00,01
1,1111000000,1,25,321.00,107.77,7.85,8.53,12.08,11.15,7.07,10.78,10.63,16.32,16.93,2.34,01
2,1111000000,1,30,349.00,117.23,17.72,12.49,10.43,12.13,7.40,18.66,13.35,15.36,9.66,8.80,01
3,1111000000,1,35,353.00,110.58,21.52,14.27,9.60,12.93,10.15,17.26,21.35,10.42,13.39,6.71,01
4,1111000000,1,40,420.00,79.58,16.60,11.60,10.83,10.16,22.11,11.11,13.40,7.52,6.29,20.09,01


### 2. groupby
    - 행정동코드, 성별, 연령대, month

In [9]:
group_df = del_interest_life_df.groupby(['month', '행정동코드', '성별', '연령대']).sum().reset_index()
group_df.head()

,month,행정동코드,성별,연령대,총인구,1인가구수,커뮤니케이션이 적은 집단,평일 외출이 적은 집단,휴일 외출이 적은 집단,출근소요시간 및 근무시간이 많은 집단,외출이 매우 적은 집단(전체),외출이 매우 많은 집단,동영상서비스 이용이 많은 집단,생활서비스 이용이 많은 집단,재정상태에 대한 관심집단,외출-커뮤니케이션이 모두 적은 집단(전체)
0,01,1111000000,1,20,6810.25,2515.20,251.09,360.00,334.28,381.79,162.71,147.94,403.51,130.27,222.07,149.94
1,01,1111000000,1,25,7420.00,2836.03,304.42,420.01,327.76,471.03,246.28,215.86,455.73,336.22,350.96,256.05
2,01,1111000000,1,30,6493.00,2221.19,245.32,274.47,239.51,254.18,175.39,345.79,252.29,277.08,226.17,171.47
3,01,1111000000,1,35,5519.00,1669.59,271.17,275.48,152.71,219.25,200.05,235.75,244.78,166.20,153.62,214.65
4,01,1111000000,1,40,6444.00,1266.64,229.37,211.30,158.36,166.79,269.81,147.28,202.70,125.32,127.16,259.53


### 3. 성별
    - 남자(1) -> 0
    - 여자(2) -> 1

In [10]:
group_df['성별'] = group_df['성별'].replace({1:0, 2:1})
group_df.head()

,month,행정동코드,성별,연령대,총인구,1인가구수,커뮤니케이션이 적은 집단,평일 외출이 적은 집단,휴일 외출이 적은 집단,출근소요시간 및 근무시간이 많은 집단,외출이 매우 적은 집단(전체),외출이 매우 많은 집단,동영상서비스 이용이 많은 집단,생활서비스 이용이 많은 집단,재정상태에 대한 관심집단,외출-커뮤니케이션이 모두 적은 집단(전체)
0,01,1111000000,0,20,6810.25,2515.20,251.09,360.00,334.28,381.79,162.71,147.94,403.51,130.27,222.07,149.94
1,01,1111000000,0,25,7420.00,2836.03,304.42,420.01,327.76,471.03,246.28,215.86,455.73,336.22,350.96,256.05
2,01,1111000000,0,30,6493.00,2221.19,245.32,274.47,239.51,254.18,175.39,345.79,252.29,277.08,226.17,171.47
3,01,1111000000,0,35,5519.00,1669.59,271.17,275.48,152.71,219.25,200.05,235.75,244.78,166.20,153.62,214.65
4,01,1111000000,0,40,6444.00,1266.64,229.37,211.30,158.36,166.79,269.81,147.28,202.70,125.32,127.16,259.53


### 4. 연령대 : 20~60대(10대 간격)
    - 연령대 컬럼의 값이 일의자리가 5인 것을 0으로 변경
    - groupby : month, 행정동코드, 성별, 연령대가 같은 것을 Sum
    - 70대 삭제

In [11]:
group_df['연령대'] = group_df['연령대'] // 10 * 10
age_group_df = group_df.groupby(['month', '행정동코드', '성별', '연령대']).sum().reset_index()
age_group_df.head()

,month,행정동코드,성별,연령대,총인구,1인가구수,커뮤니케이션이 적은 집단,평일 외출이 적은 집단,휴일 외출이 적은 집단,출근소요시간 및 근무시간이 많은 집단,외출이 매우 적은 집단(전체),외출이 매우 많은 집단,동영상서비스 이용이 많은 집단,생활서비스 이용이 많은 집단,재정상태에 대한 관심집단,외출-커뮤니케이션이 모두 적은 집단(전체)
0,01,1111000000,0,20,14230.25,5351.23,555.51,780.01,662.04,852.82,408.99,363.80,859.24,466.49,573.03,405.99
1,01,1111000000,0,30,12012.00,3890.78,516.49,549.95,392.22,473.43,375.44,581.54,497.07,443.28,379.79,386.12
2,01,1111000000,0,40,12994.00,2474.59,442.82,405.10,273.28,344.47,510.47,278.08,401.44,239.10,211.75,502.39
3,01,1111000000,0,50,12905.00,2045.24,343.46,340.01,228.36,323.20,491.97,114.93,329.21,270.24,229.29,527.90
4,01,1111000000,0,60,7890.00,1655.16,226.32,226.68,205.55,205.80,224.09,43.73,208.16,173.25,129.52,237.39


In [12]:
del_age_df = age_group_df[age_group_df['연령대'] != 70]
del_age_df.head(10)

,month,행정동코드,성별,연령대,총인구,1인가구수,커뮤니케이션이 적은 집단,평일 외출이 적은 집단,휴일 외출이 적은 집단,출근소요시간 및 근무시간이 많은 집단,외출이 매우 적은 집단(전체),외출이 매우 많은 집단,동영상서비스 이용이 많은 집단,생활서비스 이용이 많은 집단,재정상태에 대한 관심집단,외출-커뮤니케이션이 모두 적은 집단(전체)
0,01,1111000000,0,20,14230.25,5351.23,555.51,780.01,662.04,852.82,408.99,363.80,859.24,466.49,573.03,405.99
1,01,1111000000,0,30,12012.00,3890.78,516.49,549.95,392.22,473.43,375.44,581.54,497.07,443.28,379.79,386.12
2,01,1111000000,0,40,12994.00,2474.59,442.82,405.10,273.28,344.47,510.47,278.08,401.44,239.10,211.75,502.39
3,01,1111000000,0,50,12905.00,2045.24,343.46,340.01,228.36,323.20,491.97,114.93,329.21,270.24,229.29,527.90
4,01,1111000000,0,60,7890.00,1655.16,226.32,226.68,205.55,205.80,224.09,43.73,208.16,173.25,129.52,237.39
6,01,1111000000,1,20,14294.00,5686.18,439.22,605.73,600.04,635.38,223.91,405.92,524.24,477.73,406.85,224.50
7,01,1111000000,1,30,11262.11,3295.47,305.22,436.42,254.57,378.27,247.66,428.18,371.88,307.97,268.67,277.19
8,01,1111000000,1,40,13391.00,2286.17,198.85,378.56,168.88,291.87,216.08,208.51,276.22,232.01,201.59,285.04
9,01,1111000000,1,50,13304.00,1989.48,197.95,328.73,134.47,332.11,296.67,87.40,301.02,211.56,203.98,431.36
10,01,1111000000,1,60,8757.00,1809.74,174.13,320.34,156.28,273.13,249.89,32.90,253.65,170.34,147.61,316.04


### 5. 총인구수, 1인가구수
    - 소수부분 버림

In [13]:
del_age_df['총인구'] = del_age_df['총인구'].astype(int)
del_age_df['1인가구수'] = del_age_df['1인가구수'].astype(int)
final_interest_life_df = del_age_df
final_interest_life_df.head()

/tmp/ipykernel_49704/3606377079.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del_age_df['총인구'] = del_age_df['총인구'].astype(int)
/tmp/ipykernel_49704/3606377079.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del_age_df['1인가구수'] = del_age_df['1인가구수'].astype(int)


,month,행정동코드,성별,연령대,총인구,1인가구수,커뮤니케이션이 적은 집단,평일 외출이 적은 집단,휴일 외출이 적은 집단,출근소요시간 및 근무시간이 많은 집단,외출이 매우 적은 집단(전체),외출이 매우 많은 집단,동영상서비스 이용이 많은 집단,생활서비스 이용이 많은 집단,재정상태에 대한 관심집단,외출-커뮤니케이션이 모두 적은 집단(전체)
0,01,1111000000,0,20,14230,5351,555.51,780.01,662.04,852.82,408.99,363.80,859.24,466.49,573.03,405.99
1,01,1111000000,0,30,12012,3890,516.49,549.95,392.22,473.43,375.44,581.54,497.07,443.28,379.79,386.12
2,01,1111000000,0,40,12994,2474,442.82,405.10,273.28,344.47,510.47,278.08,401.44,239.10,211.75,502.39
3,01,1111000000,0,50,12905,2045,343.46,340.01,228.36,323.20,491.97,114.93,329.21,270.24,229.29,527.90
4,01,1111000000,0,60,7890,1655,226.32,226.68,205.55,205.80,224.09,43.73,208.16,173.25,129.52,237.39


In [14]:
final_interest_life_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2250 entries, 0 to 2698
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   month                    2250 non-null   object 
 1   행정동코드                    2250 non-null   int64  
 2   성별                       2250 non-null   int64  
 3   연령대                      2250 non-null   int64  
 4   총인구                      2250 non-null   int64  
 5   1인가구수                    2250 non-null   int64  
 6   커뮤니케이션이 적은 집단            2250 non-null   float64
 7   평일 외출이 적은 집단             2250 non-null   float64
 8   휴일 외출이 적은 집단             2250 non-null   float64
 9   출근소요시간 및 근무시간이 많은 집단     2250 non-null   float64
 10  외출이 매우 적은 집단(전체)         2250 non-null   float64
 11  외출이 매우 많은 집단             2250 non-null   float64
 12  동영상서비스 이용이 많은 집단         2250 non-null   float64
 13  생활서비스 이용이 많은 집단          2250 non-null   float64
 14  재정상태에 대한 관심집단            2250

### CSV로 저장

In [ ]:
final_interest_life_df.to_csv('pre_interest_life.csv', index=False)